Copyright 2023-2023 Lawrence Livermore National Security, LLC and other MuyGPyS
Project Developers. See the top-level COPYRIGHT file for details.

SPDX-License-Identifier: MIT

# Nonstationary tutorial

This notebook demonstrates how to use hierarchical nonstationary hyperparameters to perform nonstationary regression using a hierarchical model.

⚠️ _Note that this is still an experimental feature at this point._ ⚠️

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from MuyGPyS.gp import MuyGPS
from MuyGPyS.gp.distortion import IsotropicDistortion, l2, F2
from MuyGPyS.gp.hyperparameter import ScalarHyperparameter
from MuyGPyS.gp.hyperparameter.experimental import (
    sample_knots,
    HierarchicalNonstationaryHyperparameter,
)
from MuyGPyS.gp.kernels import RBF
from MuyGPyS.gp.noise import HomoscedasticNoise

We will set a random seed here for consistency when building docs.
In practice we would not fix a seed.

In [ ]:
np.random.seed(0)

## Preliminary setup

For simplicity, we start with an isotropic distortion so we only need to use a single `HierarchicalNonstationaryHyperparameter`.
Let's also build a GP with a fixed length scale for comparison.

Let's create some training data with a little bit of noise.

In [ ]:
data_max = 5
data_count = 500
train_step = 10
train_count = int(data_count / train_step)
test_count = data_count - train_count
noise_prior = 1e-5
noise_actual = 2e-4
x = np.linspace(-data_max, data_max, num=data_count)
y = np.sinc(x) - np.mean(np.sinc(x))
x = (x - np.min(x)) / (2 * np.max(x))
train_features = np.reshape(x[::train_step] + np.random.normal(scale=noise_actual, size=train_count), (-1, 1))
train_responses = np.reshape(y[::train_step] + np.random.normal(scale=noise_actual, size=train_count), (-1, 1))
test_features = x[np.mod(np.arange(data_count), train_step) != 0].reshape(test_count, 1)
test_responses = y[np.mod(np.arange(data_count), train_step) != 0].reshape(test_count, 1)

We can visualize the true function we are trying to predict, along with the training data with which will optimize a model.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7,5))
ax.plot(x, y, label="True Response")
ax.plot(train_features, train_responses, '.', label="Training Data")
plt.legend()
plt.show()

We will create a stationary MuyGPs object for reference.

In [ ]:
muygps_fixed = MuyGPS(
    kernel=RBF(
        metric=IsotropicDistortion(
            l2,
            length_scale=ScalarHyperparameter(1.0),
        ),
    ),
    eps=HomoscedasticNoise(noise_prior),
)

## Hierarchical Nonstationary MuyGPs

We will also create a hierarchical nonstationary MuyGPs object, where we assume that the `length_scale` of the distance function itself varies according to a Gaussian process with some "knots", locations in the range of the function where we assume that we know or can learn the true value of the `length_scale`. We will start by sampling some knots and giving them initial values.

In [ ]:
knot_count = 6
knot_features = sample_knots(feature_count=1, knot_count=knot_count)
knot_features = np.array(sorted(knot_features))
knot_values = np.array([2.0, 1.5, 1.0, 1.0, 1.5, 2.0])

We then create a `MuyGPS` object like before, except now we specify that the `length_scale` is hierarchical and pass the knots. 

In [ ]:
high_level_kernel = RBF(
    IsotropicDistortion(
        F2,
        length_scale=ScalarHyperparameter(1.0))
)

muygps = MuyGPS(
    kernel=RBF(
        metric=IsotropicDistortion(
            l2,
            length_scale=HierarchicalNonstationaryHyperparameter(
                knot_features, knot_values, high_level_kernel
            ),
        ),
    ),
    eps=HomoscedasticNoise(noise_prior),
)

We can visualize the knots and the resulting `length_scale` surface over the domain of the function.
Unlike `ScalarHyperparameter`, `HierarchicalNonstationaryHyperparameter` takes an array of feature vectors for each point where you would like to evaluate the local value of the hyperparameter.

In [ ]:
length_scale_curve = muygps.kernel.distortion_fn.length_scale(x.reshape(data_count, 1))

Since this is a small example, we can evaluate and display the predicted `length_scale` values across the whole domain.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7,5))
ax.set_title("Hierarchical Length Scale Surface Over the Domain")
order = np.argsort(knot_features[:,0])
ax.plot(knot_features[order,:], knot_values[order], "*", label="Knot Values")
ax.plot(x, length_scale_curve, label="Interpolated Surface")
plt.legend()
plt.show()

Now we can proceed as usual to generate the nearest neighbors lookup index and tensors.

In [ ]:
from MuyGPyS.neighbors import NN_Wrapper

nn_count = 30
nbrs_lookup = NN_Wrapper(train_features, nn_count, nn_method="exact", algorithm="ball_tree")

Note that in this simple example we're using all of the data as batch points, i.e. we're not really batching, since the dataset is very small.

In [ ]:
from MuyGPyS.gp.tensors import make_predict_tensors

test_indices = np.arange(test_count)
test_nn_indices, _ = nbrs_lookup.get_nns(test_features)

(
    test_crosswise_diffs,
    test_pairwise_diffs,
    test_nn_targets,
) = make_predict_tensors(
    test_indices,
    test_nn_indices,
    test_features,
    train_features,
    train_responses,
)

One notable difference when using a hierarchical model is that the kernel takes an additional tensor, the batch tensor, which can be easily obtained using the `batch_features_tensor` helper function.

In [ ]:
from MuyGPyS.gp.tensors import batch_features_tensor

batch_test_features = batch_features_tensor(test_features, test_indices)

Finally, we're ready to realize the kernel tensors and use them to predict the response of the test data. First using the GP with a fixed length scale.

In [ ]:
Kcross_flat_fixed = muygps_fixed.kernel(test_crosswise_diffs)
K_flat_fixed = muygps_fixed.kernel(test_pairwise_diffs)
mean_flat_fixed = muygps_fixed.posterior_mean(
    K_flat_fixed, Kcross_flat_fixed, test_nn_targets
)
var_flat_fixed = muygps_fixed.posterior_variance(
    K_flat_fixed, Kcross_flat_fixed
)

Then the hierarchical GP.

In [ ]:
Kcross_hierarchical_fixed = muygps.kernel(
    test_crosswise_diffs, batch_features=batch_test_features
)
K_hierarchical_fixed = muygps.kernel(
    test_pairwise_diffs, batch_features=batch_test_features
)
mean_hierarchical_fixed = muygps.posterior_mean(
    K_hierarchical_fixed, Kcross_hierarchical_fixed, test_nn_targets
)
var_hierarchical_fixed = muygps.posterior_variance(
    K_hierarchical_fixed, Kcross_hierarchical_fixed
)

And we can visualize the results by plotting the predicted means as well as one predicted standard deviation.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 5))

ax.plot(x, y, label="truth")
ax.plot(test_features, mean_flat_fixed, ".-", label="flat fixed")
ax.fill_between(
    np.ravel(test_features),
    np.ravel(mean_flat_fixed + np.sqrt(var_flat_fixed) * 1.96),
    np.ravel(mean_flat_fixed - np.sqrt(var_flat_fixed) * 1.96),
    facecolor="C1",
    alpha=0.2,
)
ax.plot(test_features, mean_hierarchical_fixed, "--", label="hierarchical fixed")
ax.fill_between(
    np.ravel(test_features),
    np.ravel(mean_hierarchical_fixed + np.sqrt(var_hierarchical_fixed) * 1.96),
    np.ravel(mean_hierarchical_fixed - np.sqrt(var_hierarchical_fixed) * 1.96),
    facecolor="C2",
    alpha=0.2,
)
plt.legend()
plt.show()

## Optimization

The knot values of hierarchical nonstationary hyperparameters can be optimized using like any other hyperparameters, using the `optimize_from_tensors` utility. But first, we need to initialize them as `ScalarHyperparameter`s with bounds rather than as fixed values.

In [ ]:
bounds = [0.1, 5.0]
knot_values_to_be_optimized = [ScalarHyperparameter("sample", bounds) for _ in range(knot_count)]

Let's recreate a MuyGPs object. It's identical to the one we've created before except for the knot values.

In [ ]:
hierarchical_to_be_optimized = MuyGPS(
    kernel=RBF(
        metric=IsotropicDistortion(
            l2,
            length_scale=HierarchicalNonstationaryHyperparameter(
                knot_features, knot_values_to_be_optimized, high_level_kernel
            ),
        ),
    ),
    eps=HomoscedasticNoise(noise_prior),
)

Then we use `make_train_tensors` to obtain the training tensors. Once again, we use all the training data instead of batching it due to the small size of the dataset.

In [ ]:
from MuyGPyS.gp.tensors import make_train_tensors

train_indices = np.arange(train_count)
train_nn_indices, _ = nbrs_lookup.get_nns(train_features)

(
    batch_crosswise_diffs,
    batch_pairwise_diffs,
    batch_targets,
    batch_nn_targets,
) = make_train_tensors(
    train_indices,
    train_nn_indices,
    train_features,
    train_responses,
)

batch_train_features = batch_features_tensor(train_features, train_indices)

Then we use `optimize_from_tensors` to create a new MuyGPs object for which the knot values have been fixed to their optimal values. Note that we must pass the `batch_features` tensor, this time from the training data.

⚠️ _The next four cells are for testing purposes and should be removed prior to finalizing this tutorial._ ⚠️

In [ ]:
from MuyGPyS.optimize.loss import get_loss_func
from MuyGPyS.optimize.objective import make_obj_fn, make_kernels_fn, make_raw_predict_and_loss_fn, make_var_predict_and_loss_fn
from MuyGPyS.optimize.sigma_sq import make_sigma_sq_optim
from MuyGPyS.optimize.utils import _switch_on_loss_method

loss_method = "lool"
sigma_method = "analytic"
loss_fn = get_loss_func(loss_method)
kernel_fn = hierarchical_to_be_optimized.kernel.get_opt_fn()
mean_fn = hierarchical_to_be_optimized.get_opt_mean_fn()
var_fn = hierarchical_to_be_optimized.get_opt_var_fn()
sigma_sq_fn = make_sigma_sq_optim(sigma_method, hierarchical_to_be_optimized)

kernels_fn = make_kernels_fn(
    kernel_fn,
    batch_pairwise_diffs,
    batch_crosswise_diffs,
)

predict_and_loss_fn = _switch_on_loss_method(
    loss_method,
    make_raw_predict_and_loss_fn,
    make_raw_predict_and_loss_fn,
    make_var_predict_and_loss_fn,
    make_raw_predict_and_loss_fn,
    loss_fn,
    mean_fn,
    var_fn,
    sigma_sq_fn,
    batch_nn_targets,
    batch_targets,
)

In [ ]:
K, Kcross = kernels_fn(batch_features=batch_train_features)

In [ ]:
obj_fn = make_obj_fn(
    "loo_crossval",
    loss_method,
    loss_fn,
    kernel_fn,
    mean_fn,
    var_fn,
    sigma_sq_fn,
    batch_pairwise_diffs,
    batch_crosswise_diffs,
    batch_nn_targets,
    batch_targets,
    batch_features=batch_train_features,
)

In [ ]:
import inspect

print(f"kernel_fn: {inspect.signature(kernel_fn)}")
print(f"kernels_fn: {inspect.signature(kernels_fn)}")
print(f"mean_fn: {inspect.signature(mean_fn)}")
print(f"var_fn: {inspect.signature(var_fn)}")
print(f"predict_and_loss_fn: {inspect.signature(predict_and_loss_fn)}")
print(f"obj_fn: {inspect.signature(obj_fn)}")

In [ ]:
obj_fn()

In [ ]:
from MuyGPyS.optimize import optimize_from_tensors



In [ ]:
from MuyGPyS.optimize import optimize_from_tensors

batch_train_features = batch_features_tensor(train_features, train_indices)

hierarchical_optimized = optimize_from_tensors(
    hierarchical_to_be_optimized,
    batch_targets,
    batch_nn_targets,
    batch_crosswise_diffs,
    batch_pairwise_diffs,
    loss_method=loss_method,
    obj_method="loo_crossval",
    opt_method="bayes",
    verbose=True,
    batch_features=batch_train_features,
    init_points=20,
    n_iter=20,
    allow_duplicate_points=True,
)

The optimized model has set new knot values, which we can use to visualize the learned `length_scale` surface. 

In [ ]:
hierarchical_optimized.kernel.distortion_fn.length_scale._knot_values()

In [ ]:
length_scale_curve_optimized = hierarchical_optimized.kernel.distortion_fn.length_scale(x.reshape(data_count, 1))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7,5))
ax.set_title("Trained Hierarchical Length Scale Surface Over the Domain")
order = np.argsort(knot_features[:,0])
ax.plot(
    knot_features[order,:], 
    hierarchical_optimized.kernel.distortion_fn.length_scale._knot_values()[order], 
    "*", 
    label="Knot Values")
ax.plot(x, length_scale_curve_optimized, label="Interpolated Surface")
plt.legend()
plt.show()

We can now use the optimized the kernel to predict the test responses.

In [ ]:
Kcross_hierarchical_opt = hierarchical_optimized.kernel(test_crosswise_diffs, batch_test_features)
K_hierarchical_opt = hierarchical_optimized.kernel(test_pairwise_diffs, batch_test_features)
mean_hierarchical_opt = hierarchical_optimized.posterior_mean(
    K_hierarchical_opt, Kcross_hierarchical_opt, test_nn_targets
)
var_hierarchical_opt = hierarchical_optimized.posterior_variance(
    K_hierarchical_opt, Kcross_hierarchical_opt
)

We also optimize a flat model with the same batch for comparison.

In [ ]:
flat_to_be_optimized = MuyGPS(
    kernel=RBF(
        metric=IsotropicDistortion(
            l2,
            length_scale=ScalarHyperparameter("sample", [0.001, 5.0]),
        ),
    ),
    eps=HomoscedasticNoise(noise_prior),
)

In [ ]:
flat_optimized = optimize_from_tensors(
    flat_to_be_optimized,
    batch_targets,
    batch_nn_targets,
    batch_crosswise_diffs,
    batch_pairwise_diffs,
    loss_method=loss_method,
    obj_method="loo_crossval",
    opt_method="bayes",
    verbose=True,
    allow_duplicate_points=True,
)

In [ ]:
Kcross_flat_opt = flat_optimized.kernel(test_crosswise_diffs)
K_flat_opt = flat_optimized.kernel(test_pairwise_diffs)
mean_flat_opt = flat_optimized.posterior_mean(
    K_flat_opt, Kcross_flat_opt, test_nn_targets
)
var_flat_opt = flat_optimized.posterior_variance(
    K_flat_opt, Kcross_flat_opt
)

And visualize the results.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 5))

ax.plot(x, y, label="truth")
ax.plot(test_features, mean_flat_fixed, ".-", label="flat fixed")
ax.fill_between(
    np.ravel(test_features),
    np.ravel(mean_flat_fixed + np.sqrt(var_flat_fixed) * 1.96),
    np.ravel(mean_flat_fixed - np.sqrt(var_flat_fixed) * 1.96),
    facecolor="C1",
    alpha=0.2,
)
ax.plot(test_features, mean_hierarchical_fixed, "--", label="hierarchical fixed")
ax.fill_between(
    np.ravel(test_features),
    np.ravel(mean_hierarchical_fixed + np.sqrt(var_hierarchical_fixed) * 1.96),
    np.ravel(mean_hierarchical_fixed - np.sqrt(var_hierarchical_fixed) * 1.96),
    facecolor="C2",
    alpha=0.2,
)
ax.plot(test_features, mean_flat_opt, "--", label="flat optimized")
ax.fill_between(
    np.ravel(test_features),
    np.ravel(mean_flat_opt + np.sqrt(var_flat_opt) * 1.96),
    np.ravel(mean_flat_opt - np.sqrt(var_flat_opt) * 1.96),
    facecolor="C3",
    alpha=0.2,
)
ax.plot(test_features, mean_hierarchical_opt, "--", label="hierarchical optimized")
ax.fill_between(
    np.ravel(test_features),
    np.ravel(mean_hierarchical_opt + np.sqrt(var_hierarchical_opt) * 1.96),
    np.ravel(mean_hierarchical_opt - np.sqrt(var_hierarchical_opt) * 1.96),
    facecolor="C3",
    alpha=0.2,
)
plt.legend()
plt.show()